In [2]:
from selenium import webdriver
from base64 import b64encode
import sys
import re
from urllib.request import urlopen
from html import unescape
import openpyxl
import datetime
import time
from selenium.webdriver.chrome.options import Options
import chromedriver_binary
# selenium 4.3.0から
from selenium.webdriver.common.by import By

def get_athletes_htmls_by_selenium_chrome(url):
  options = Options()
# 必要ならPROXYを設定する
#  PROXY = "127.0.0.1:8080"
#  options.add_argument('--proxy-server=http://%s' % PROXY)  
# ブラウザはヘッドレス（表示しない）とする
#  options.add_argument('--headless')
  options.add_argument('--window-size=1280,1024')

# Chromeのドライバを得る
  driver = webdriver.Chrome(options=options)
  driver.implicitly_wait = 60  # pageをloadするまでの待ち時間を設定
  driver.get(url)              # URLへアクセス
  driver.implicitly_wait = 60  # 待ち時間を設定
# selenium 4.3.0 でfind_elements_by_css_selectorがなくなり、以下に変更された
#  elements = driver.find_elements_by_css_selector('tr.table-row--hover')
  elements = driver.find_elements(By.CSS_SELECTOR, 'tr.table-row--hover')
  driver.implicitly_wait = 60  # 待ち時間を設定

  html_list=[]

#  for i in range(len(elements)):
  for i in range(3):
    driver.get(url)              # URLへアクセス
    driver.implicitly_wait = 90   # 待ち時間を設定
# selenium 4.3.0 でfind_elements_by_css_selectorがなくなり、以下に変更された
#    elements = driver.find_elements_by_css_selector('tr.table-row--hover')
    elements = driver.find_elements(By.CSS_SELECTOR, 'tr.table-row--hover')
    driver.implicitly_wait = 90
    time.sleep(10)
    try:
      elements[i].click()
      driver.implicitly_wait = 60  # 待ち時間を設定
      time.sleep(10)
#      print(driver.page_source)
      html_list.append(driver.page_source)    # アクセスしたサイトのページソースを返す
#    close_elements=driver.find_elements_by_css_selector('button.close')
#    close_elements[1].click()
    except:
      continue
  driver.quit()
#  print(html_list)
  return(html_list)

def get_athlete_data(html, rownum):

#  print(html)
  a = re.search(r'<!-- Media modal -->(.*?)<!-- Media modal -->', html, re.DOTALL).group(1)
  d = re.search(r'<h2 (.*?)/h2>', a, re.DOTALL).group(1)
  athlete_name = re.search(r'>(.*?)<', d, re.DOTALL).group(1)
  sheet.cell(row=rownum, column=1).value = athlete_name
  athlete_name_rownum = rownum
#  rownum = rownum + 1
  
  for b in re.findall(r'<tr (.*?)</tr>', a, re.DOTALL):
#    print('b', b)
    col = 2
    for c in re.findall(r'<td data-bind=(.*?)/td>', b, re.DOTALL):
#      print('c', c)
      d = re.search(r'>(.*?)<', c, re.DOTALL).group(1)
      sheet.cell(row=rownum, column=col).value = d
#      print('d', d)
      col = col +1
    rownum = rownum + 1
#  print(a)
  i = 14
  for e in re.findall(r'<span (.*?)/span>', a, re.DOTALL):
#    print('e', e)
    sheet.cell(row=athlete_name_rownum, column=i).value = re.search(r'>(.*?)<', e, re.DOTALL).group(1)
    i = i + 1
  return(rownum)

for i in range(1):
  url = 'https://www.worldathletics.org/world-rankings/overall-ranking/men?regionType=countries&region=jpn&page='+ str(i+1)

  html_list = get_athletes_htmls_by_selenium_chrome(url)

  rownum = 1
  wb = openpyxl.Workbook()
  sheet = wb.active
  sheet.cell(row=rownum, column=1).value = 'ATHLETE'
  sheet.cell(row=rownum, column=2).value = 'DATE'
  sheet.cell(row=rownum, column=3).value = 'COMPETITION'
  sheet.cell(row=rownum, column=4).value = 'CNT.'
  sheet.cell(row=rownum, column=5).value = 'CAT.'
  sheet.cell(row=rownum, column=6).value = 'EVENT'
  sheet.cell(row=rownum, column=7).value = 'TYPE'
  sheet.cell(row=rownum, column=8).value = 'PL.'
  sheet.cell(row=rownum, column=9).value = 'RESULT'
  sheet.cell(row=rownum, column=10).value = 'R.SC'
  sheet.cell(row=rownum, column=11).value = 'WR'
  sheet.cell(row=rownum, column=12).value = 'PL.SC'
  sheet.cell(row=rownum, column=13).value = 'PF.SC'
  sheet.cell(row=rownum, column=14).value = 'Average of Performance Scores'
  sheet.cell(row=rownum, column=15).value = 'Ranking Score'

  rownum = rownum + 1

  for html in html_list:
    rownum = get_athlete_data(html, rownum)
  
  now = datetime.datetime.now()
  wb.save('worldathletics_japanese_athletes_data_' + "{0:%Y-%m-%d}".format(now) + '_' + str(i+1)+ '.xlsx')


